### Урок 4. Парсинг HTML. XPath

* Выберите веб-сайт с табличными данными, который вас интересует.
* Напишите код Python, использующий библиотеку requests для отправки HTTP GET-запроса на сайт и получения HTML-содержимого страницы.
* Выполните парсинг содержимого HTML с помощью библиотеки lxml, чтобы извлечь данные из таблицы.
* Сохраните извлеченные данные в CSV-файл с помощью модуля csv.

Ваш код должен включать следующее:

- Строку агента пользователя в заголовке HTTP-запроса, чтобы имитировать веб-браузер и избежать блокировки сервером.
- Выражения XPath для выбора элементов данных таблицы и извлечения их содержимого.
- Обработка ошибок для случаев, когда данные не имеют ожидаемого формата.
- Комментарии для объяснения цели и логики кода.

Целью данной работы будет попытка получить данные по турнирам настольного тенниса от Лига-Про.

1. Пытаемся получить ответ с get-запроса по интересующему нас дню
2. Получить информацию по этому дню о предстоящих турнирах со списком участников
3. Сформировать из полученных данных датафрейм
4. Сохранить данных в табличном формате в файле csv

In [138]:
#!pip install lxml

In [139]:
# импортируем нужные библиотеки
from lxml import html
import requests
from pprint import pprint
import csv
import pandas as pd

In [140]:
url='https://tt.sport-liga.pro/'

# задаем user-agent
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
    }
# в params надо указать интересующую нас дату в формате YYYY:MM:DD
param = {
    'year': '2024',
    'month': '05',
    'day': '7'
}

url_tour = 'tours/'
session = requests.Session()

In [141]:
# просьба ввести данные пользователя из консоли (В РАЗРАБОТКЕ)

# year, month, day = input('Введите дату в формате YYYY:MM:DD: ').split('.')
# params = {
#     'year': year,
#     'month': month,
#     'day': day
# }

In [142]:
# делаем get-запрос
response = session.get(url+url_tour, params=param, headers=header)
# проверяем статус-код
if response.status_code < 400:
    # если да, печатаем статус код и представляем ответ в виде дерева html
    print(response.status_code)
    tree = html.fromstring(response.text)
else:
    print('Что-то пошло не так')

200


Необходимые данные находятся на странице в нижней таблице. Также для сбора фамилий участников турнира нам нужно переходить на его страницу (ее можно "достать" из таблицы.)

In [143]:
# находим нужную таблицу на странице
table = tree.xpath('//table[@class="table"]//tr')
# находим ссылки на переход к просмотру турнира
ref = tree.xpath('.//td[@class="tournament-name"]/a/@href')

Для объединения всех данных в один список будем проходиться циклом for сразу по двум спискам, для этого объединим их функцией zip

In [144]:
tournaments = []
for row, item in zip(table[1:], ref):
    # сделаем get-запрос на страницу турнира
    res = session.get(url+item, headers=header)
    # и получим дерево из его ответа
    tour = html.fromstring(res.text)
    # создадим словарик для каждого турнира
    tours_info = {}

    # из даты нам нужно только время начало турнира
    date = row.xpath('.//td[@class="tournament-date"]/text()')
    date = [el.split()[-1] for el in date][0]

    # из названия турнира мы возьмем
    tours = row.xpath('.//td[@class="tournament-name"]/a/text()')
    # название стола
    tables = [el.split()[1][:-1] for el in tours][0]
    # уровень лиги
    liga = [el.split()[3] for el in tours][0]

    # участников турнира получим из страницы турнира
    parties = tour.xpath('//td[@class="nowrap"]/a/text()')

    tours_info['date'] = date
    tours_info['tables'] = tables
    tours_info['liga'] = liga
    tours_info['parties'] = parties
    
    # добавим каждый словарь в список
    tournaments.append(tours_info)


In [145]:
pprint(tournaments)

[{'date': '23:45',
  'liga': '550-600',
  'parties': ['Солдатов И',
              'Ильин А',
              'Кириленко А',
              'Никулин С',
              'Сухарников Н'],
  'tables': 'А6'},
 {'date': '23:45',
  'liga': '400-450',
  'parties': ['Беспалов Н', 'Ермолов Д', 'Кутин В', 'Федоров А', 'Михайлов И'],
  'tables': 'А4'},
 {'date': '23:30',
  'liga': '450-500',
  'parties': ['Тарасенков Н',
              'Попов О',
              'Зюганов О',
              'Зеленский Р',
              'Чернов В'],
  'tables': 'А5'},
 {'date': '20:00',
  'liga': '500-550.',
  'parties': ['Нехведович В', 'Ляльков В', 'Жук А', 'Атоманчик К'],
  'tables': 'А15'},
 {'date': '20:00',
  'liga': '250-300',
  'parties': ['Агапов Н', 'Егоров П', 'Щавелев Н', 'Смирнов М'],
  'tables': 'А9'},
 {'date': '20:00',
  'liga': '600-700',
  'parties': ['Гиревенков А', 'Богатов Д', 'Петроченко Д', 'Ляличев Д'],
  'tables': 'А3'},
 {'date': '19:45',
  'liga': '800-900',
  'parties': ['Пищук Д', 'Баталов А', 'И

Запишем полученную информацию в csv-файл

In [146]:
with open('./tournaments.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=tournaments[0].keys())
    writer.writeheader
    for row in tournaments:
        writer.writerow(row)
print('Данные успешно записаны')

Данные успешно записаны


Для наглядности представим наш список в виде датафрейма

In [147]:
df = pd.DataFrame(tournaments)
df

,date,tables,liga,parties
0,23:45,А6,550-600,"[Солдатов И, Ильин А, Кириленко А, Никулин С, ..."
1,23:45,А4,400-450,"[Беспалов Н, Ермолов Д, Кутин В, Федоров А, Ми..."
2,23:30,А5,450-500,"[Тарасенков Н, Попов О, Зюганов О, Зеленский Р..."
3,20:00,А15,500-550.,"[Нехведович В, Ляльков В, Жук А, Атоманчик К]"
4,20:00,А9,250-300,"[Агапов Н, Егоров П, Щавелев Н, Смирнов М]"
5,20:00,А3,600-700,"[Гиревенков А, Богатов Д, Петроченко Д, Ляличе..."
6,19:45,А6,800-900,"[Пищук Д, Баталов А, Инназаров А, Денисов А-м]"
7,19:45,А4,500-550,"[Вашков А, Пандур И, Пучинин А, Мягков С]"
8,19:30,А5,350-400,"[Степанов И, Земит И, Самоха Е, Мельников М]"
9,16:00,А3,450-500,"[Фролов А Г, Игнатов М, Семенов П, Чернов В]"


Вывод: В результате проделанной работы мы получаем датафрейм, содержащий нужную информацию. Этот датафрейм планируется использовать для подбора турнира с более предпочтительными игроками.

Также планируется исследовать каждый турнир на предмет статистики личных встреч между игроками

In [148]:
#df = df[df['tables'] != 'А15']
df[df['date'] > '15:00']

,date,tables,liga,parties
0,23:45,А6,550-600,"[Солдатов И, Ильин А, Кириленко А, Никулин С, ..."
1,23:45,А4,400-450,"[Беспалов Н, Ермолов Д, Кутин В, Федоров А, Ми..."
2,23:30,А5,450-500,"[Тарасенков Н, Попов О, Зюганов О, Зеленский Р..."
3,20:00,А15,500-550.,"[Нехведович В, Ляльков В, Жук А, Атоманчик К]"
4,20:00,А9,250-300,"[Агапов Н, Егоров П, Щавелев Н, Смирнов М]"
5,20:00,А3,600-700,"[Гиревенков А, Богатов Д, Петроченко Д, Ляличе..."
6,19:45,А6,800-900,"[Пищук Д, Баталов А, Инназаров А, Денисов А-м]"
7,19:45,А4,500-550,"[Вашков А, Пандур И, Пучинин А, Мягков С]"
8,19:30,А5,350-400,"[Степанов И, Земит И, Самоха Е, Мельников М]"
9,16:00,А3,450-500,"[Фролов А Г, Игнатов М, Семенов П, Чернов В]"
